<a href="https://colab.research.google.com/github/aiden213/Deep-Learning/blob/master/Seismic_Classifiers_Pub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tensorflow-gpu==2.3.0

     |████████████████████████████████| 320.4MB 50kB/s 


In [3]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
 raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6716042898854728006, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 9927295753382729350
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 4613394800182111354
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11133970048
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 1200412819387810251
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

for cpu and ram info:

In [6]:
!cat /proc/cpuinfo
!cat /proc/meminfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2300.000
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs
bogomips	: 4600.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	:

First we'll import all the libraries we need down the line. We also set the "random seed", so results can be reproduced by avid readers. Keras should report using the Tensorflow backend, otherwise reproducibility cannot be guaranteed.

In [10]:
!pip install obspy

     |████████████████████████████████| 24.7MB 129kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for obspy: filename=obspy-1.2.2-cp36-cp36m-linux_x86_64.whl size=21666241 sha256=70f612e4b67fcb503e51c36535bd36b1328ff8c7f2ed6957209331b248519afb
  Stored in directory: /root/.cache/pip/wheels/94/e0/86/44fd4cea7661f42431c8f6d030f2758ff275cccffcbe8fa2b8
Successfully built obspy


In [19]:
import numpy as np
import pandas as pd
import obspy        #additional module
import time
import matplotlib.pyplot as plt

import tqdm
from tqdm import tnrange, tqdm_notebook

from obspy.io.segy.segy import _read_segy
from sklearn.model_selection import train_test_split

np.random.seed(42)
%matplotlib notebook


For experimentation with network models, we keep the keras imports separate, to reduce loading time.

In [22]:
from tensorflow.keras.models import Sequential, Model, clone_model
from tensorflow.keras.layers import Conv2D, Dense, Activation, Flatten, Dropout, Input, BatchNormalization

#from tensorflow.keras.layers.normalization import BatchNormalization  tf2.3"BatchNormalization"直接在layers下了

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD


We need to define some parameters. As we are using Transfer learning, we have to adjust these parameters to fit into the network that we use and test. 

| Model   |      Channels      |  Patch-Size |
|----------|:-------------:|------:|
| Waldeland |  1 | 64 |
| VGG16 |    3   |  64 |
| ResNet50 | 3 | 244 |

In [ ]:
patch_size = 64 # for ResNet50 put 244
batch_size = 256
num_channels = 1
num_classes = 9
all_examples = 158812
num_examples = 7500
epochs = 20
steps=450
sampler = list(range(all_examples))

opt = 'adam'
lossfkt = ['categorical_crossentropy']
metrica = ['mae', 'acc']

Here we test, whether we are running on CPU or GPU. We want to run on GPU, if it's not in the device list. It will be slow.

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
# It should say GPU here. Otherwise your model will run sloooow.

# Data Loading
Now let's load the F3 data and read three slices. The labeled data, as well as, a distal inline and a crossline.

In [ ]:
filename = 'data/Dutch Government_F3_entire_8bit seismic.segy'

t0=time.time()
stream0 = _read_segy(filename, headonly=True)
print('--> data read in {:.1f} sec'.format(time.time()-t0)) #Thanks to aadm 

t0=time.time()

labeled_data = np.stack(t.data for t in stream0.traces if t.header.for_3d_poststack_data_this_field_is_for_in_line_number == 339).T
inline_data = np.stack(t.data for t in stream0.traces if t.header.for_3d_poststack_data_this_field_is_for_in_line_number == 500).T
xline_data = np.stack(t.data for t in stream0.traces if t.header.for_3d_poststack_data_this_field_is_for_cross_line_number == 500).T

print('--> created slices in {:.1f} sec'.format(time.time()-t0))

# Helper Functions
From these slices, we need to extract patches. While, we could do that before and save them as array or image data, using a generator that utilizes the CPU, while the GPU trains the network is a bit more storage- and memory-friendly. `patch_extractor2D()` automates the patch-extraction and pads sides, where necessary.

Then we build `acc_assess()` to format our test accuracy assessment nicely, because we're lazy and retyping it for every model we build is a nuisance.

All functions are accompanied with a little sanity check. While this is not automated testing (like TDD), it does help to make sure, our function works as intended.

In [ ]:
def patch_extractor2D(img,mid_x,mid_y,patch_size,dimensions=1):
    try:
        x,y,c = img.shape
    except ValueError:
        x,y = img.shape
        c=1
    patch= np.pad(img, patch_size//2, 'constant', constant_values=0)[mid_y:mid_y+patch_size,mid_x:mid_x+patch_size] #because it's padded we don't subtract half patches all the tim
    if c != dimensions:
        tmp_patch = np.zeros((patch_size,patch_size,dimensions))
        for uia in range(dimensions):
            tmp_patch[:,:,uia] = patch
        return tmp_patch
    return patch
image=np.random.rand(10,10)//.1
print(image)

patch_extractor2D(image,10,10,4,1)

In [ ]:
def acc_assess(data,loss=['categorical_crossentropy'],metrics=['acc']):
    if not isinstance(loss, list):
        try:
            loss = [loss]
        except:
            raise("Loss must be list.")
    if not isinstance(metrics, list):
        try:
            metrics = [metrics]
        except:
            raise("Metrics must be list.")
    out='The test loss is {:.3f}\n'.format(data[0])
    for i, metric in enumerate(metrics):            
        if metric in 'mae':
            out += "The total mean error on the test is {:.3f}\n".format(data[i+1])
        if metric in 'accuracy':
            out += "The test accuracy is {:.1f}%\n".format(data[i+1]*100)
    return out
print(acc_assess([1,2,3],'bla',["acc", "mae"]))

# Exploratory Data Analysis
We need to load and check our labels. 

In [ ]:
labels = pd.read_csv('data/classification.ixz', delimiter=" ", names=["Inline","Xline","Time","Class"])
labels.describe()

In [ ]:
labels["Xline"]-=300-1
labels["Time"] = labels["Time"]//4
labels.describe()

In [ ]:
labeled_data.shape

In [ ]:
fig2 = plt.figure(figsize=(15.0, 10.0))
vml = np.percentile(labeled_data, 99)
img1 = plt.imshow(labeled_data, cmap="Greys", vmin=-vml, vmax=vml, aspect='auto')
plt.yticks(np.arange(0, 462, 100), np.arange(0, 462*4, 400))
plt.xlabel('Trace Location')
plt.ylabel('Time [ms]')
plt.savefig('labeled_data.png', bbox_inches='tight')
plt.show()

In [ ]:
fig2 = plt.figure(figsize=(15.0, 10.0))
vmx = np.percentile(xline_data, 99)
plt.imshow(xline_data, cmap="Greys", vmin=-vmx, vmax=vmx, aspect='auto')
plt.yticks(np.arange(0, 462, 100), np.arange(0, 462*4, 400))
plt.xlabel('Trace Location')
plt.ylabel('Time [ms]')
plt.savefig('xline_data.png', bbox_inches='tight')
plt.show()

In [ ]:
fig2 = plt.figure(figsize=(15.0, 10.0))
vmy = np.percentile(inline_data, 99)
plt.imshow(inline_data, cmap="Greys", vmin=-vmy, vmax=vmy, aspect='auto')
plt.yticks(np.arange(0, 462, 100), np.arange(0, 462*4, 400))
plt.xlabel('Trace Location')
plt.ylabel('Time [ms]')
plt.savefig('inline_data.png', bbox_inches='tight')
plt.show()

In [ ]:
fig2 = plt.figure(figsize=(15.0, 10.0))
img2 = plt.imshow(labeled_data, cmap="Greys", vmin=-vml, vmax=vml, aspect='auto')
img1 = plt.scatter(labels["Xline"],labels[["Time"]],c=labels[["Class"]],cmap='Dark2',alpha=0.03)
plt.yticks(np.arange(0, 462, 100), np.arange(0, 462*4, 400))
plt.xlabel('Trace Location')
plt.ylabel('Time [ms]')
plt.savefig('label.png', bbox_inches='tight')
plt.show()

# Train the Network
Now we perform a test-train split. Then we can validate the results of our experiment.

In [ ]:
train_data, test_data, train_samples, test_samples = train_test_split(
    labels, sampler, random_state=42)
print(train_data.shape,test_data.shape)

This is the `keras` data generator that wraps the `patch_extractor2D()`.

In [ ]:
class SeismicSequence(keras.utils.Sequence):
    def __init__(self, img, x_set, t_set, y_set, patch_size, batch_size, dimensions):
        self.slice = img
        self.X,self.t = x_set,t_set
        self.batch_size = batch_size
        self.patch_size = patch_size
        self.dimensions = dimensions
        self.label = y_set
    
    def __len__(self):
        return len(self.X) // self.batch_size
    
    def __getitem__(self,idx):
        sampler = np.random.permutation(len(self.X))
        samples = sampler[idx*self.batch_size:(idx+1)*self.batch_size]
        labels = keras.utils.to_categorical(self.label[samples], num_classes=9)
        if self.dimensions == 1:
            return np.expand_dims(np.array([patch_extractor2D(self.slice,self.X[x],self.t[x],self.patch_size,self.dimensions) for x in samples]), axis=4), labels
        else:
            return np.array([patch_extractor2D(self.slice,self.X[x],self.t[x],self.patch_size,self.dimensions) for x in samples]), labels

We define several callbacks for keras. The training should be stopped early, if the validation loss or the categorical cross entropy do not improve within the defined patience. Checkpoints are written to `tmp.h5` for every epoch.

In [ ]:
earlystop1 = keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=3,
                              verbose=0, mode='auto')

earlystop2 = keras.callbacks.EarlyStopping(monitor='val_acc',
                              min_delta=0,
                              patience=3,
                              verbose=0, mode='auto')

checkpoint = keras.callbacks.ModelCheckpoint('tmp.h5', 
                                     monitor='val_loss', 
                                     verbose=0, 
                                     save_best_only=False, 
                                     save_weights_only=False, 
                                     mode='auto', 
                                     period=1)

callbacklist = [TQDMNotebookCallback(leave_inner=True, leave_outer=True), earlystop1, earlystop2, checkpoint]

## Waldeland CNN
The model introduced by Waldeland, reproduced from MalenoV. Compared to today's standards this is a relatively shallow CNN. We train the network from scratch.

In [ ]:
tf.logging.set_verbosity(tf.logging.ERROR)

model_vanilla = Sequential()
model_vanilla.add(Conv2D(50, (5, 5), padding='same', input_shape=(patch_size,patch_size,1), strides=(4, 4), data_format="channels_last",name = 'conv_layer1'))
model_vanilla.add(BatchNormalization())
model_vanilla.add(Activation('relu'))
model_vanilla.add(Conv2D(50, (3, 3), strides=(2, 2), padding = 'same',name = 'conv_layer2'))
model_vanilla.add(Dropout(0.5))
model_vanilla.add(BatchNormalization())
model_vanilla.add(Activation('relu'))
model_vanilla.add(Conv2D(50, (3, 3), strides=(2, 2), padding= 'same',name = 'conv_layer3'))
model_vanilla.add(Dropout(0.4))
model_vanilla.add(BatchNormalization())
model_vanilla.add(Activation('relu'))
model_vanilla.add(Conv2D(50, (3, 3), strides=(2, 2), padding= 'same',name = 'conv_layer4'))
model_vanilla.add(Dropout(0.2))
model_vanilla.add(BatchNormalization())
model_vanilla.add(Activation('relu'))
model_vanilla.add(Conv2D(50, (3, 3), strides=(2, 2), padding= 'same',name = 'conv_layer5'))
model_vanilla.add(Flatten())
model_vanilla.add(Dense(50,name = 'dense_layer1'))
model_vanilla.add(BatchNormalization())
model_vanilla.add(Activation('relu'))
model_vanilla.add(Dense(10,name = 'attribute_layer'))
model_vanilla.add(BatchNormalization())
model_vanilla.add(Activation('relu'))
model_vanilla.add(Dense(num_classes, name = 'pre-softmax_layer'))
model_vanilla.add(BatchNormalization())
model_vanilla.add(Activation('softmax'))

model_vanilla.compile(loss=lossfkt,
                  optimizer=opt,
                  metrics=metrica)

In [ ]:
t0=time.time()

hist_vanilla = model_vanilla.fit_generator(
    SeismicSequence(
        labeled_data,
        train_data["Xline"].values,
        train_data["Time"].values,
        train_data["Class"].values,
        patch_size,
        batch_size,
        1),
    steps_per_epoch=steps,
    validation_data = SeismicSequence(
        labeled_data,
        test_data["Xline"].values,
        test_data["Time"].values,
        test_data["Class"].values,
        patch_size,
        batch_size,
        1),
    validation_steps = len(test_samples)//batch_size,
    epochs = epochs,
    verbose = 0,
    callbacks = callbacklist)

print('--> Training for Waldeland CNN took {:.1f} sec'.format(time.time()-t0)) #Thanks to aadm 

In [ ]:
model_vanilla.save("vanilla_model.h5")

In [ ]:
vanillascore=model_vanilla.evaluate(np.expand_dims(np.array([patch_extractor2D(labeled_data,labels["Xline"][x],labels["Time"][x],64) for x in test_samples]), axis=4),keras.utils.to_categorical(labels["Class"][test_samples], num_classes=9), verbose=0)
print(acc_assess(vanillascore,lossfkt,metrica))

Looking at the metric on training as well as validation gives a good overview, if we are doing appropriate training or if we are overfitting.

In [ ]:
print(hist_vanilla.history.keys())
plt.plot(hist_vanilla.history['acc'])
plt.plot(hist_vanilla.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(hist_vanilla.history['loss'])
plt.plot(hist_vanilla.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
t_max, y_max = xline_data.shape

half_patch = patch_size//2

predx = np.full_like(xline_data,-1)

for space in tqdm_notebook(range(y_max),desc='Space'):
    for depth in tqdm_notebook(range(t_max),leave=False, desc='Time'):
        predx[depth,space] = np.argmax(model_vanilla.predict(np.expand_dims(np.expand_dims(patch_extractor2D(xline_data,space,depth,patch_size), axis=0), axis=4)))

In [ ]:
np.save('vanilla_predx.npy',predx,allow_pickle=False)

In [ ]:
plt.imshow(predx)

In [ ]:
fig2 = plt.figure(figsize=(15.0, 10.0))
img2 = plt.imshow(xline_data, cmap="Greys", vmin=-vmx, vmax=vmx, aspect='auto')
img1 = plt.imshow(predx, aspect='auto', cmap="Dark2", alpha=0.5)
plt.savefig('pred1_x.png', bbox_inches='tight')
plt.show()

In [ ]:
t_max, y_max = inline_data.shape

half_patch = patch_size//2

predi= np.full_like(inline_data,-1)

for space in tqdm_notebook(range(y_max),desc='Space'):
    for depth in tqdm_notebook(range(t_max),leave=False, desc='Time'):
        predi[depth,space] = np.argmax(model_vanilla.predict(np.expand_dims(np.expand_dims(patch_extractor2D(inline_data,space,depth,patch_size), axis=0), axis=4)))

In [ ]:
np.save('vanilla_predi.npy',predi,allow_pickle=False)

In [ ]:
predi = np.load('vanilla_predi.npy')
plt.imshow(predi)

In [ ]:
np.save('vanilla_predi.npy',predi,allow_pickle=False)
fig2 = plt.figure(figsize=(15.0, 10.0))
img2 = plt.imshow(inline_data, cmap="Greys", vmin=-vmy, vmax=vmy, aspect='auto')
img1 = plt.imshow(predi, aspect='auto', cmap="Dark2", alpha=0.5)
plt.yticks(np.arange(0, 462, 100), np.arange(0, 462*4, 400))
plt.xlabel('Trace Location')
plt.ylabel('Time [ms]')
plt.savefig('pred1_i.png', bbox_inches='tight')
plt.show()

## VGG16 Transfer Learning
We import the VGG16 model trained on the ImageNet dataset. We freeze all layers and cut off the classification part. We can then retrain the classification neurons, to see if the filters generalize to seismic data.

In [ ]:
from keras.applications.vgg16 import VGG16
from keras import backend as K
K.set_image_dim_ordering('tf')   

In [ ]:
input_tensor = Input(shape=(patch_size,patch_size,3))
base_model = keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor, input_shape=None)

for layer in base_model.layers[:8]:
    layer.trainable = False

In [ ]:
x = base_model.output
x = Flatten()(x)
x = Dense(256,name = 'dense_layer1')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(.5)(x)
x = Dense(num_classes, name = 'pre-softmax_layer')(x)
x = BatchNormalization()(x)
x = Activation('softmax')(x)

vgg = Model(input=base_model.input, output=x)

In [ ]:
sgd = SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)
vgg.compile(loss=lossfkt,
                  optimizer=sgd,
                  metrics=metrica)

In [ ]:
t0 = time.time()

vgg_hist = vgg.fit_generator(
    SeismicSequence(
        labeled_data,
        train_data["Xline"].values,
        train_data["Time"].values,
        train_data["Class"].values,
        patch_size,
        batch_size,
        3),
    steps_per_epoch=steps,
    validation_data = SeismicSequence(
        labeled_data,
        test_data["Xline"].values,
        test_data["Time"].values,
        test_data["Class"].values,
        patch_size,
        batch_size,
        3),
    validation_steps = len(test_data)//batch_size,
    epochs = epochs,
    verbose = 0,
    callbacks = callbacklist)

print('--> Training for VGG transfer took {:.1f} sec'.format(time.time()-t0))

In [ ]:
vgg.save('vgg_model.h5')

In [ ]:
vggscore=vgg.evaluate(np.array([patch_extractor2D(labeled_data,labels["Xline"][x],labels["Time"][x],64,3) for x in test_samples]), keras.utils.to_categorical(labels["Class"][test_samples], num_classes=9))
print(acc_assess(vggscore,lossfkt,metrica))

In [ ]:
print(hist_vanilla.history.keys())
plt.plot(vgg_hist.history['acc'])
plt.plot(vgg_hist.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(vgg_hist.history['loss'])
plt.plot(vgg_hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
t_max, y_max = xline_data.shape

half_patch = patch_size//2

vgg_predx = np.full_like(xline_data,-1)

for space in tqdm_notebook(range(y_max),desc='Space'):
    for depth in tqdm_notebook(range(t_max),leave=False, desc='Time'):
        vgg_predx[depth,space] = np.argmax(vgg.predict(np.expand_dims(patch_extractor2D(xline_data,space,depth,patch_size,3), axis=0)))

In [ ]:
np.save('vgg_predx.npy',vgg_predx,allow_pickle=False)

In [ ]:
plt.imshow(vgg_predx)

In [ ]:
vgg_predx=np.load('vgg_predx.npy')
fig2 = plt.figure(figsize=(15.0, 10.0))
img2 = plt.imshow(xline_data, cmap="Greys", vmin=-vmx, vmax=vmx, aspect='auto')
img1 = plt.imshow(vgg_predx, aspect='auto', cmap="Dark2", alpha=0.5)
plt.yticks(np.arange(0, 462, 100), np.arange(0, 462*4, 400))
plt.xlabel('Trace Location')
plt.ylabel('Time [ms]')
plt.savefig('vgg1_x.png', bbox_inches='tight')
plt.show()

In [ ]:
t_max, y_max = inline_data.shape

half_patch = patch_size//2

vgg_predi = np.full_like(inline_data,-1)

for space in tqdm_notebook(range(y_max),desc='Space'):
    for depth in tqdm_notebook(range(t_max),leave=False, desc='Time'):
        vgg_predi[depth,space] = np.argmax(vgg.predict(np.expand_dims(patch_extractor2D(inline_data,space,depth,patch_size,3), axis=0)))

In [ ]:
np.save('vgg_predi.npy',vgg_predi,allow_pickle=False)

In [ ]:
plt.imshow(vgg_predi)

In [ ]:
vgg_predi= np.load('vgg_predi.npy')
fig2 = plt.figure(figsize=(15.0, 10.0))
img2 = plt.imshow(inline_data, cmap="Greys", vmin=-vmy, vmax=vmy, aspect='auto')
img1 = plt.imshow(vgg_predi, aspect='auto', cmap="Dark2", alpha=0.5)
plt.yticks(np.arange(0, 462, 100), np.arange(0, 462*4, 400))
plt.xlabel('Trace Location')
plt.ylabel('Time [ms]')
plt.savefig('vgg1_i.png', bbox_inches='tight')
plt.show()

## ResNet50 Transfer Learning
We import the ResNet50 that was trained on the ImageNet data and freeze all layers, like we did for the VGG16. Then we retrain the classifier to see if the learned filters generalize on seismic data.

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras import backend as K
K.set_image_dim_ordering('tf')   
patch_size=244

In [ ]:
input_tensor = Input(shape=(patch_size,patch_size,3))
res_base = ResNet50(include_top=False, weights='imagenet', input_tensor=input_tensor, input_shape=None, pooling=None)

for layer in res_base.layers[:45]:
    layer.trainable = False

In [ ]:
q = res_base.output
q = Flatten()(q)
q = BatchNormalization()(q)
q = Activation('relu')(q)
q = Dense(10,name = 'attribute_layer')(q)
q = BatchNormalization()(q)
q = Activation('relu')(q)
q = Dense(num_classes, name = 'pre-softmax_layer')(q)
q = BatchNormalization()(q)
q = Activation('softmax')(q)
resnet = Model(input=res_base.input, output=q)

In [ ]:
sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
resnet.compile(loss=lossfkt,
                  optimizer=opt,
                  metrics=metrica)

In [ ]:
t0 = time.time()

batch_size=50
res_hist = resnet.fit_generator(
    SeismicSequence(
        labeled_data,
        train_data["Xline"].values,
        train_data["Time"].values,
        train_data["Class"].values,
        patch_size,
        batch_size,
        3),
    steps_per_epoch=steps,
    validation_data = SeismicSequence(
        labeled_data,
        test_data["Xline"].values,
        test_data["Time"].values,
        test_data["Class"].values,
        patch_size,
        batch_size,
        3),
    validation_steps = len(test_data)//batch_size,
    epochs = epochs,
    verbose = 0,
    callbacks = callbacklist)

print('--> Training for ResNet transfer took {:.1f} sec'.format(time.time()-t0))

In [ ]:
resnet.save('resnet_model.h5')

In [ ]:
resnetscore=resnet.evaluate(np.array([patch_extractor2D(labeled_data,labels["Xline"][x],labels["Time"][x],patch_size,3) for x in test_samples]), keras.utils.to_categorical(labels["Class"][test_samples], num_classes=9))
print(acc_assess(resnetscore,lossfkt,metrica))

In [ ]:
t_max, y_max = xline_data.shape

half_patch = patch_size//2

resnet_predx = np.full_like(xline_data,-1)

for space in tqdm_notebook(range(y_max),desc='Space'):
    for depth in tqdm_notebook(range(t_max),leave=False, desc='Time'):
        resnet_predx[depth,space] = np.argmax(resnet.predict(np.expand_dims(patch_extractor2D(xline_data,space,depth,patch_size,3), axis=0)))

In [ ]:
np.save('resnet_predx.npy',resnet_predx,allow_pickle=False)

In [ ]:
plt.imshow(resnet_predx)

In [ ]:
fig2 = plt.figure(figsize=(15.0, 10.0))
img2 = plt.imshow(xline_data, cmap="Greys", vmin=-vmx, vmax=vmx, aspect='auto')
img1 = plt.imshow(resnet_predx, aspect='auto', cmap="Dark2", alpha=0.8)
plt.savefig('resnet_x.png', bbox_inches='tight')
plt.show()

In [ ]:
t_max, y_max = inline_data.shape

half_patch = patch_size//2

resnet_predi = np.full_like(inline_data,-1)

for space in tqdm_notebook(range(y_max-400,y_max-300),desc='Space'):
    for depth in tqdm_notebook(range(t_max-400,t_max-300),leave=False, desc='Time'):
        resnet_predi[depth,space] = np.argmax(resnet.predict(np.expand_dims(patch_extractor2D(inline_data,space,depth,patch_size,3), axis=0)))

In [ ]:
np.save('resnet_predi.npy',resnet_predi,allow_pickle=False)

In [ ]:
plt.imshow(resnet_predi)

In [ ]:
fig2 = plt.figure(figsize=(15.0, 10.0))
img2 = plt.imshow(inline_data, cmap="Greys", vmin=-vmy, vmax=vmy, aspect='auto')
img1 = plt.imshow(resnet_predi, aspect='auto', cmap="Dark2", alpha=0.8)
plt.savefig('resnet_i.png', bbox_inches='tight')
plt.show()

In [ ]:
print(res_hist.history.keys())
plt.plot(res_hist.history['acc'])
plt.plot(res_hist.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(res_hist.history['loss'])
plt.plot(res_hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plot_model(resnet, to_file='model_resnet.png')
plot_model(resnet, to_file='model_resnet_shapes.png', show_shapes=True)
SVG(model_to_dot(resnet).create(prog='dot', format='svg'))

# Model Summary
We can see the summaries of the layers in the model definitions. Leveraging high-dimensional CNNs that are already trained can be very valuable.

In [ ]:
model_vanilla.summary()

In [ ]:
vgg.summary()

In [ ]:
resnet.summary()